In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv("Amazon_Unlocked_Mobile.csv")

In [3]:
df.head()

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
0,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,I feel so LUCKY to have found this used (phone...,1.0
1,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,"nice phone, nice up grade from my pantach revu...",0.0
2,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,Very pleased,0.0
3,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,It works good but it goes slow sometimes but i...,0.0
4,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,Great phone to replace my lost phone. The only...,0.0


In [5]:
#To drop any row with missing values we use .dropna(inplace=True)
df.dropna(inplace=True)

In [11]:
#To copy a dataframe where no rating is equal to 3 we do this
df=df[df["Rating"]!=3]
df["Positive"]=np.where(df["Rating"]>3,1,0)
df.head()

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes,Positive
0,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,I feel so LUCKY to have found this used (phone...,1.0,1
1,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,"nice phone, nice up grade from my pantach revu...",0.0,1
2,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,Very pleased,0.0,1
3,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,It works good but it goes slow sometimes but i...,0.0,1
4,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,Great phone to replace my lost phone. The only...,0.0,1


In [15]:
len(df[df["Positive"]==1])

230674

In [16]:
from sklearn.model_selection import train_test_split

In [18]:
X_train, X_test, y_train, y_test = train_test_split(df["Reviews"], df["Positive"], test_size=0.33, random_state=101)

In [19]:
print(X_train.shape)

(206545,)


In [20]:
from sklearn.feature_extraction.text import CountVectorizer
cm=CountVectorizer()

In [21]:
vect=cm.fit(X_train)


In [22]:
print(type(vect))

<class 'sklearn.feature_extraction.text.CountVectorizer'>


In [23]:
vect


CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [26]:
len(vect.get_feature_names())

51356

In [27]:
X_train_vect=vect.transform(X_train)

In [30]:
X_train_vect

<206545x51356 sparse matrix of type '<class 'numpy.int64'>'
	with 5476986 stored elements in Compressed Sparse Row format>

In [31]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression()

In [32]:
model.fit(X_train_vect,y_train)

C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [34]:
from sklearn.metrics import roc_auc_score,accuracy_score
pred=model.predict(vect.transform(X_test))
print("The Auc score is {}".format(roc_auc_score(y_test,pred)))

The Auc score is 0.9263552426757401


In [35]:
print("The accuracy is {}".format(accuracy_score(y_test,pred)))

The accuracy is 0.9506153422718515


In [44]:
feature_name=np.array(vect.get_feature_names())
feature_name

array(['00', '000', '0000', ..., '这不是欧美版的', '非常赞', '좋군'], dtype='<U156')

In [43]:
#to get the coefficient index within the feature_name array we need to sort the coef
coe_index=model.coef_[0].argsort()
print("Smallest Coef {}".format(feature_name[coe_index[:10]]))
print("Biggest Coef {}".format(feature_name[coe_index[:-11:-1]]))

Smallest Coef ['worst' 'false' 'unusable' 'worthless' 'dissatisfied' 'junk' 'nope'
 'dieing' 'unsatisfied' 'horrible']
Biggest Coef ['excelent' 'excelente' 'excellent' 'exelente' 'loving' 'amazing' 'loves'
 'love' 'perfecto' 'lovely']


In [45]:
from sklearn.feature_extraction.text import  TfidfVectorizer
vecct=TfidfVectorizer(min_df=5).fit(X_train)

In [48]:
X_train_vecc=vecct.transform(X_train)
model.fit(X_train_vecc,y_train)
pred1=model.predict(vecct.transform(X_test))
print("The accuracy is {}".format(accuracy_score(y_test,pred1)))

C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


The accuracy is 0.9483348405614752
